In [4]:
import os
os.environ["JAX_PLATFORM_NAME"] = "cpu"
import netket as nk
import numpy as np
import matplotlib.pyplot as plt
import netket.nn as nknn
import flax.linen as nn
import jax.numpy as jnp
import jax

In [2]:
J = [1, 0.2];L = 8;edge_colors = []
for i in range(L):
    edge_colors.append([i, (i+1)%L, 1])
    edge_colors.append([i, (i+2)%L, 2])
g = nk.graph.Graph(edges=edge_colors)
sigmaz = [[1, 0], [0, -1]];  mszsz = (np.kron(sigmaz, sigmaz))
exchange = np.asarray([[0, 0, 0, 0], [0, 0, 2, 0], [0, 2, 0, 0], [0, 0, 0, 0]])
bond_operator = [
        (J[0] * mszsz).tolist(),    (J[1] * mszsz).tolist(),
        (-J[0] * exchange).tolist(),(J[1] * exchange).tolist(),
        ]
bond_color = [1, 2, 1, 2];   sigmaz = [[1, 0], [0, -1]]
mszsz = (np.kron(sigmaz, sigmaz));
exchange = np.asarray([[0, 0, 0, 0], [0, 0, 2, 0], [0, 2, 0, 0], [0, 0, 0, 0]])
bond_operator = [
        (J[0] * mszsz).tolist(),    (J[1] * mszsz).tolist(),
        (-J[0] * exchange).tolist(),(J[1] * exchange).tolist(),
    ]

bond_color = [1, 2, 1, 2]
hi = nk.hilbert.Spin(s=0.5, total_sz=0.0, N=g.n_nodes)
op = nk.operator.GraphOperator(hi, graph=g, bond_ops=bond_operator, bond_ops_colors=bond_color)

In [5]:
alpha = 2
model = nk.models.RBM(alpha=alpha)
rng_key = jax.random.PRNGKey(1234)
initial_state = jnp.ones((1, L))
model.init(rng_key, initial_state)  


{'params': {'Dense': {'kernel': Array([[ 8.23246599e-03, -1.62925398e-03, -2.40223347e-03,
            2.78062964e-05,  1.30084155e-03,  2.83382276e-02,
           -2.25602852e-02, -2.15331842e-02,  3.44524503e-03,
            5.50811133e-03,  1.20296895e-02,  4.54389130e-04,
           -5.77788075e-03,  8.21401924e-03,  1.69800268e-02,
           -7.77806707e-04],
          [ 1.37063644e-02,  4.87131036e-03,  1.14309609e-03,
            3.08117845e-04,  9.29583768e-03, -1.50667115e-03,
           -3.47820230e-03, -9.00833316e-03, -1.02926909e-02,
           -1.28183274e-02, -1.12155781e-02,  1.02980578e-04,
           -1.58269342e-02, -1.49403211e-02,  5.70996898e-03,
           -4.56203223e-03],
          [-4.97841698e-03,  1.92195686e-02,  8.99691817e-03,
           -1.78639157e-02, -1.70784178e-02, -1.05293558e-03,
           -7.15695959e-03, -1.77260158e-02, -5.08216048e-04,
            1.82142206e-02, -1.80235861e-03,  3.81051873e-03,
           -1.95666178e-04,  1.26472993e-02, 

In [6]:
it = 100

In [7]:
sa = nk.sampler.MetropolisExchange(hilbert=hi, graph=g, d_max = 2)
vs = nk.vqs.MCState(sa, model, n_samples=1008)
vs_i = vs;
opt = nk.optimizer.Sgd(learning_rate=0.01)
sr = nk.optimizer.SR(diag_shift=0.01)
gs = nk.VMC(hamiltonian=op, optimizer=opt, variational_state=vs, preconditioner=sr)
# We need to specify the local operators as a matrix acting on a local Hilbert space 
sf = []
sites = []
structure_factor = nk.operator.LocalOperator(hi, dtype=complex)
for i in range(0, L):
    for j in range(0, L):
        structure_factor += (nk.operator.spin.sigmaz(hi, i)*nk.operator.spin.sigmaz(hi, j))*((-1)**(i-j))/L

gs.run(out='test', n_iter=it, obs={'Structure Factor': structure_factor}) 
print("------------  -------------")
vs_f = vs

100%|█| 100/100 [00:02<00:00, 42.77it/s, Energy=-13.40369 ± 0.00091 [σ²=0.00083,

------------  -------------


In [ ]:
rng_key = jax.random.PRNGKey(1234)
initial_state = jnp.ones((1, L))
model.init(rng_key, initial_state)  


In [ ]:
gs.state.parameters

In [ ]:
vs_i.parameters

In [ ]:
vs.parameters